[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1njzOgHWgguSuA732RYROJDiCua3hGO_R?usp=sharing)

# Introduction to Structured Responses for Healthcare Data

This script demonstrates how to use **structured responses** with DeepSeek Models (V3 and R1) on Fireworks AI to generate reliable and structured healthcare data.


### Why Use Structured Responses?
- **Predictability**: Enforces medical data constraints to ensure logical outputs.
- **Automation**: Structured outputs can be used for patient record generation, automated reports, and clinical decision support.
- **Integration**: JSON and Grammar Mode allow seamless integration into electronic health record (EHR) systems and research databases.

### What You'll Learn
1. **Basic API Setup**: Connecting to Fireworks AI and using DeepSeek v3 and R1.
2. **Grammar Mode**: Generating structured healthcare data within predefined constraints.
3. **JSON Mode**: Ensuring machine-readable structured responses.
4. **Reasoning JSON Mode**: Capturing the model's thought process alongside structured responses.

### Setup

Install all the dependencies and import the required python modules.

In [19]:
!pip3 install --upgrade fireworks-ai pydantic python-dotenv > /dev/null

### Setup your API Key

In order to use the DeepSeek models, you must first obtain Fireworks API Keys. If you don't already have one, create a [Fireworks AI](https://fireworks.ai) account. Under Account Settings, click on [API Keys](https://fireworks.ai/api-keys) to create one.

In [20]:
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv  # Import dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Fireworks client using API key from .env
client = Fireworks(api_key=os.getenv("FIREWORKS_API_KEY"))

# 0️⃣ **Basic API Call to DeepSeek v3**

### Chat Completions API
This is a simple test call to ensure the Fireworks API is working correctly.

In [21]:
model_name = "accounts/fireworks/models/deepseek-v3"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "What are the latest advancements in AI for healthcare?"},
    ],
)
print(response.choices[0].message.content)

The field of AI in healthcare is rapidly evolving, with numerous advancements that are transforming diagnosis, treatment, patient care, and administrative processes. Here are some of the latest advancements as of 2023:

---

### 1. **AI in Medical Imaging and Diagnostics**
   - **Improved Accuracy in Radiology**: AI algorithms are now being integrated into radiology workflows, enhancing the detection of diseases like cancer, fractures, and neurological disorders (e.g., Alzheimer’s) in X-rays, MRIs, and CT scans.
   - **Point-of-Care Ultrasound (POCUS)**: AI-powered portable ultrasound devices are enabling faster and more accurate diagnoses in emergency settings.
   - **Ophthalmology**: AI systems like Google's DeepMind are being used to detect diabetic retinopathy and macular degeneration from retinal scans.

---

### 2. **Generative AI in Drug Discovery**
   - **Accelerating Drug Development**: AI models like AlphaFold by DeepMind and others are predicting protein structures, speeding

# 1️⃣ **Grammar Mode with BNF for Healthcare Data**

### **What is Grammar Mode?**
Grammar Mode ensures that the model outputs responses that conform to a strict structure, preventing unexpected variations in responses. This is particularly useful for generating structured data, such as medical records.

#### **How Grammar Mode Works**
- Uses a **GBNF (Grammar-Based Normal Form)** to constrain the model's output.
- Ensures the output adheres to predefined rules for valid healthcare data.

Read more about [GBNF](https://github.com/ggerganov/llama.cpp/tree/master/grammars), and how to [use it](https://docs.fireworks.ai/structured-responses/structured-output-grammar-based).


In [22]:
# Define the grammar to enforce structured output for healthcare reports
healthcare_grammar = """
root ::= patient-record

patient-record ::= "Patient Name: " name " Age: " age " Gender: " gender " Diagnosis: " diagnosis " Treatment: " treatment " Medications: " medications " Notes: " notes

name ::= ("John Doe" | "Jane Smith" | "Alice Johnson" | "Bob Brown")
age ::= ("25" | "30" | "40" | "50" | "60")
gender ::= ("Male" | "Female" | "Non-binary")
diagnosis ::= ("Hypertension" | "Diabetes Type 2" | "Asthma" | "Coronary Artery Disease" | "Osteoarthritis")
treatment ::= ("Lifestyle Changes" | "Physical Therapy" | "Surgery" | "Medication Management")
medications ::= ("Metformin" | "Lisinopril" | "Albuterol" | "Atorvastatin" | "Ibuprofen")
notes ::= ("Patient shows good adherence to treatment." | "Further tests required." | "Improvement noted in recent follow-up.")
"""

In [23]:
# Call the model using Grammar Mode to enforce structured output
response_grammar = client.chat.completions.create(
    model=model_name,
    response_format={"type": "grammar", "grammar": healthcare_grammar},
    messages=[
        {"role": "user", "content": "Generate a structured healthcare patient record."}
    ],
    max_tokens=3000,
)

In [24]:
# Extract structured output from Grammar Mode
grammar_output = response_grammar.choices[0].message.content
print("🔹 Grammar Mode Output:", grammar_output)

🔹 Grammar Mode Output: Patient Name: John Doe Age: 40 Gender: Male Diagnosis: Hypertension Treatment: Lifestyle Changes Medications: Lisinopril Notes: Patient shows good adherence to treatment.


# 2️⃣ **JSON Mode for Healthcare Data**

JSON mode is a subset of grammar mode and is a feature that allows you to specify a JSON schema to force a language model to respond in a structured JSON format. This has several benefits:

* Clarity and precision in responses
* Easy parsing and processing of output
* Improved integration with healthcare systems
* Standardized format for consistent data handling

In [25]:
from pydantic import BaseModel, Field
from typing import List

In [26]:
class PatientRecord(BaseModel):
    name: str
    age: int
    gender: str
    diagnosis: str
    treatment: str
    medications: str
    notes: str


In [27]:
# API Call - JSON Mode
response_json = client.chat.completions.create(
    model=model_name,
    response_format={"type": "json_object", "schema": PatientRecord.model_json_schema()},
    messages=[
        {"role": "user", "content": "Generate a structured healthcare record for a patient diagnosed with diabetes."},
    ]
)
print(response_json.choices[0].message.content)

{"name":"Anupriya Ashokan","age":45,"gender":"Female","diagnosis":"Type 2 Diabetes Mellitus","treatment":"Metformin 500 mg twice daily, lifestyle modifications","medications": "Metformin 500 mg twice daily, Lisinopril 10 mg once daily, Atorvastatin 20 mg once daily","notes":"Patient advised to monitor blood glucose levels regularly and follow a diabetic diet. Scheduled for follow-up in 3 months."}


# 3️⃣ **Reasoning Model JSON Mode for Computer Specifications**

# 3️⃣ **Reasoning Model JSON Mode for Healthcare Data**

### 🧠 **Reasoning Model JSON Mode for Healthcare Data**  

#### 🔹 **What is Reasoning JSON Mode?**  
Reasoning JSON Mode extends structured JSON generation by adding an **explanation of the model’s decision-making process**. This is done by generating a **reasoning section enclosed within `<think>...</think>` tags**, followed by a structured JSON response.

This approach enhances transparency and interpretability by providing insight into **why** the model made certain clinical recommendations.

#### 🔹 **Why Use Reasoning JSON Mode for Healthcare Data?**  
For tasks like **clinical documentation, medical decision support, or patient record validation**, Reasoning JSON Mode provides:

* ✅ **Explainability** → Understand why certain diagnoses and treatments were selected.  
* ✅ **Debugging Capability** → Identify potential inconsistencies in AI-generated medical recommendations.  
* ✅ **Structured Data** → JSON output is ready for integration into healthcare databases.


In [28]:
import json
import re
from pydantic import BaseModel
from openai import OpenAI

In [29]:
# Prepare the user input
user_input = "Generate a structured healthcare record with reasoning for a patient diagnosed with hypertension."

# Construct the messages payload
messages = [{"role": "user", "content": user_input}]

# Make the API call to the model
response = client.chat.completions.create(
    model="accounts/fireworks/models/deepseek-r1",
    messages=messages,
    response_format={"type": "json_object", "schema": PatientRecord.model_json_schema()},
    max_tokens=3000,
)

# Extract the content of the response
response_content = response.choices[0].message.content
print("show response content", response_content)

# Extract the reasoning part enclosed in <think>...</think> tags
reasoning_match = re.search(r"<think>(.*?)</think>", response_content, re.DOTALL)
reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."

# Extract the JSON part that follows after the reasoning
json_match = re.search(r"</think>\s*(\{.*\})", response_content, re.DOTALL)
json_str = json_match.group(1).strip() if json_match else "{}"

# Parse the JSON string directly into a Pydantic model
patient_record = PatientRecord.model_validate_json(json_str)

# Output the reasoning and the parsed JSON data
print("Reasoning:")
print(reasoning)
print("\nPatient Record (JSON Data):")
print(patient_record.model_dump_json(indent=4))


show response content <think>
Okay, I need to generate a structured healthcare record for a patient with hypertension, including the reasoning behind each section. Let me start by recalling the typical components of a medical record. There's usually patient demographics, chief complaint, history of present illness, past medical history, etc. 

First, I'll outline the basic structure. The user gave an example that starts with Patient Demographics, so maybe follow that example. I need to make sure each section is clear and comprehensive. 

For each section, I need to include the necessary details and then the reasoning. For instance, under Chief Complaint, the patient would mention symptoms like headache or fatigue. The reasoning would explain why those symptoms are relevant to hypertension.

In the History of Present Illness, I should include the onset, duration, and progression of symptoms. Maybe the patient noticed symptoms a few weeks ago and they've been getting worse. Also, trigger

# **Conclusion**



To learn more, visit [Fireworks AI Docs](https://docs.fireworks.ai) or join our [Discord community](https://discord.gg/fireworks-ai).